In [1]:
import numpy as np
import pandas as pd

In [7]:
train_actions = pd.read_parquet('../data/raw/train_actions.pq')

In [8]:
train_actions.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6580936 entries, 0 to 6580935
Data columns (total 6 columns):
 #   Column       Dtype         
---  ------       -----         
 0   user_id      object        
 1   loc_user_id  object        
 2   action       int64         
 3   date         datetime64[us]
 4   products     object        
 5   pageId       object        
dtypes: datetime64[us](1), int64(1), object(4)
memory usage: 301.3+ MB


In [9]:
train_actions.head()

,user_id,loc_user_id,action,date,products,pageId
0,6fd49b56-8cc6-11ed-86e0-002590c0647c,c6e357dc-121d-449d-a744-e9a0b56c2380,7,2024-09-07 00:00:04,[],2571824865
1,6fd49b56-8cc6-11ed-86e0-002590c0647c,c6e357dc-121d-449d-a744-e9a0b56c2380,7,2024-09-07 00:00:08,[],3834364438
2,f9c498ec-5d3b-11ef-86e0-002590c0647c,120c9064-1131-4dc3-8048-44184531b42e,7,2024-09-07 00:00:08,[],2448628415
3,59386b5c-e64f-11ec-8086-002590c82437,f0745572-893f-4e50-bc52-5af47badff5a,7,2024-09-07 00:00:11,[],3875013967
4,badbd396-6cab-11ef-86e0-002590c0647c,73423d85-d47c-4332-8155-5200615302b5,7,2024-09-07 00:00:11,[],3025531174
